In [26]:
import datetime
SPRING = 0
SUMMER = 1
FALL = 2
WINTER = 3
mapping_months = {
        'Jan': 1,
        'Feb': 2,
        'Mar': 3,
        'Apr': 4,
        'May': 5,
        'Jun':6,
        'Jul':7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12
}
def get_season(date, HEMISPHERE='north'):

    md = date.month * 100 + date.day

    if ((md > 320) and (md < 621)):
        s = 0 #spring
    elif ((md > 620) and (md < 923)):
        s = 1 #summer
    elif ((md > 922) and (md < 1223)):
        s = 2 #fall
    else:
        s = 3 #winter

    if not HEMISPHERE == 'north':
        s = (s + 2) % 3
    return s


def get_part_of_day(date):
    h = date.hour

    return (
        "morning"
        if 5 <= h <= 11
        else "afternoon"
        if 12 <= h <= 17
        else "evening"
        if 18 <= h <= 22
        else "night"
    )

In [35]:
# Using readlines()
file1 = open('dataset_tsmc2014\dataset_TSMC2014_NYC.txt', 'r')
file2 = open('dataset_tsmc2014\dataset_TSMC2014_TKY.txt', 'r')
with open('dataset_tsmc2014\dataset_TSMC2014_NYC.txt') as f:
   file1_lines = sum(1 for _ in f)
lines = file1.readlines() + file2.readlines()

count = 0
check_ins = {}
poi_coos = {}
poi_cat = {}
normalized_vid = {}
user_time_prefs_count = {}
poi_time_prefs_count = {}
user_season_prefs_count = {}
poi_season_prefs_count = {}
users = []
num_venues = 0 

for line in lines:
    line_split = line.split()
    if count == file1_lines:
        users_file_1 = len(users)
    uid = int(line_split[0]) - 1 if count < file1_lines else int(line_split[0]) - 1 + users_file_1
    vid = line_split[1]
    cid = line_split[2]
    for i in range(4,9):
        if line_split[i].replace('.','',1).replace('-','',1).isdigit():
            lat = line_split[i]
            lng = line_split[i+1]
            break
    

    date_info = []
    for i in range(1,8):
        #print(line_split[-i])
        date_info.append(line_split[-i])
    # 3 - Apr, 2012
    #print(date_info)
    #print(date_info[3], date_info[4], date_info[0])

    current_time = date_info[2].split(":")
    #print(current_time)
    date_normalized = datetime.datetime( int(date_info[0]), mapping_months[date_info[4]], int(date_info[3])) + \
         datetime.timedelta(hours=int(current_time[0]), minutes= int(current_time[1]), seconds=int(current_time[2])) + \
         datetime.timedelta(hours=int(date_info[6])//60, minutes= int(date_info[6]) % 60)
    
    #print(date_normalized)
    totd = get_part_of_day(date_normalized)
    # SEASONS
    season = get_season(date_normalized)


    #coords,cat
    if vid not in normalized_vid:
        normalized_vid[vid] = vid = num_venues

        poi_coos[vid]= (lat,lng)
        poi_cat[vid]= cid
        num_venues +=1
    else:
        vid = normalized_vid[vid]

    #check ins
    if(uid,vid) in check_ins:
        check_ins[(uid,vid)] += 1
    else:
        check_ins[(uid,vid)] = 1

    if uid not in users:
        users.append(uid)
    
    #user time prefs
    if uid not in user_time_prefs_count:
        user_time_prefs_count[uid] = {}
    if totd not in user_time_prefs_count[uid]:
        user_time_prefs_count[uid][totd] = 1
    else:
        user_time_prefs_count[uid][totd] += 1

    #poi time prefs
    if vid not in poi_time_prefs_count:
        poi_time_prefs_count[vid] = {}
    if totd not in poi_time_prefs_count[vid]:
        poi_time_prefs_count[vid][totd] = 1
    else:
        poi_time_prefs_count[vid][totd] += 1

        #user time prefs
    if uid not in user_season_prefs_count:
        user_season_prefs_count[uid] = {}
    if season not in user_season_prefs_count[uid]:
        user_season_prefs_count[uid][season] = 1
    else:
        user_season_prefs_count[uid][season] += 1

    #poi time prefs
    if vid not in poi_season_prefs_count:
        poi_season_prefs_count[vid] = {}
    if season not in poi_season_prefs_count[vid]:
        poi_season_prefs_count[vid][season] = 1
    else:
        poi_season_prefs_count[vid][season] += 1
    #if count == 100:
    #    break
    count+= 1
num_user = len(users)
#print(num_user, len(check_ins))
#print(user_time_prefs)

In [36]:
import operator
#max(stats.iteritems(), key=operator.itemgetter(1))[0]
user_time_prefs = {}
poi_time_prefs = {}
user_season_prefs = {}
poi_season_prefs = {}
for i,s in user_time_prefs_count.items():
    user_time_prefs[i] = max(s.items(), key=operator.itemgetter(1))[0]
for i,s in poi_time_prefs_count.items():
    poi_time_prefs[i] = max(s.items(), key=operator.itemgetter(1))[0]
for i,s in user_season_prefs_count.items():
    user_season_prefs[i] = max(s.items(), key=operator.itemgetter(1))[0]
for i,s in poi_season_prefs_count.items():
    poi_season_prefs[i] = max(s.items(), key=operator.itemgetter(1))[0]
#print(user_season_prefs)
#print(poi_season_prefs)


In [37]:
import random
with open('2_MGMPFM/new_data/fsq_data_size.txt', 'w') as f:
    f.write(str(num_user) + ' ' + str(num_venues))

with open('2_MGMPFM/new_data/fsq_pois_coos.txt', 'w') as f:
    for i,s in poi_coos.items():
        f.write(str(i) + ' ' + s[0] + ' ' + s[1] + '\n')

with open('2_MGMPFM/new_data/fsq_pois_cats.txt', 'w') as f:
    for i,s in poi_cat.items():
        f.write(str(i) + ' ' + s + '\n')

with open('2_MGMPFM/new_data/fsq_user_totd.txt', 'w') as f:
    for i,s in user_time_prefs.items():
        f.write(str(i) + ' ' + s  + '\n')
with open('2_MGMPFM/new_data/fsq_poi_totd.txt', 'w') as f:
    for i,s in poi_time_prefs.items():
        f.write(str(i) + ' ' + s  + '\n')
with open('2_MGMPFM/new_data/fsq_user_season.txt', 'w') as f:
    for i,s in user_season_prefs.items():
        f.write(str(i) + ' ' + str(s)  + '\n')
with open('2_MGMPFM/new_data/fsq_poi_season.txt', 'w') as f:
    for i,s in poi_season_prefs.items():
        f.write(str(i) + ' ' + str(s)  + '\n')

l = list(check_ins.items())
random.shuffle(l)
check_ins = dict(l)
print(len(check_ins))
cnt = 0
with open('2_MGMPFM/new_data/fsq_test.txt', 'w') as f1:
    with open('2_MGMPFM/new_data/fsq_train.txt', 'w') as f2:
        with open('2_MGMPFM/new_data/fsq_tune.txt', 'w') as f3:
            for i,s in check_ins.items():
                if cnt< 250000:
                    f1.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(s) + '\n')
                elif cnt < 280000:
                    f2.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(s) + '\n')
                else:
                    f3.write(str(i[0]) + ' ' + str(i[1]) + ' ' + str(s) + '\n')
                cnt +=1



302979
